In [21]:
import pyspark 
print(print(pyspark.__version__)) 

2.4.4.dev0
None

In [22]:
spark.conf.set(
  "fs.azure.account.key.wagssynap.blob.core.windows.net",
  "FsnZrS+XU+CObmL7yT2SvdLo4UCzR7JvgdW+HLPJVKLIOu+bnHBdBMaa09sHI6eUXeEPVhk3eGFCYd25chG+3g==")

In [23]:
df = spark.read.option("inferSchema","true").json("wasbs://telemetry@wagssynap.blob.core.windows.net/landing/*.json")

In [24]:
df.printSchema()

root
 |-- ambient_pressure: double (nullable = true)
 |-- ambient_temperature: double (nullable = true)
 |-- machineID: string (nullable = true)
 |-- pressure: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- speed_desired: long (nullable = true)
 |-- temperature: double (nullable = true)
 |-- ts: string (nullable = true)

In [28]:
display(df.head(10))

In [25]:
df.createOrReplaceTempView("telemetry")

In [26]:
%%sql
select count(*) from telemetry

In [29]:
%%sql
select min(ts), max(ts) from telemetry

In [ ]:
df.write.parquet("wasbs://telemetry@wagssynap.blob.core.windows.net/raw/telemetry")

In [31]:
dfParquet = spark.read.parquet("wasbs://telemetry@wagssynap.blob.core.windows.net/raw/telemetry")

In [32]:
dfParquet.cache()

DataFrame[ambient_pressure: double, ambient_temperature: double, machineID: string, pressure: double, speed: double, speed_desired: bigint, temperature: double, ts: string]

In [33]:
dfParquet.createOrReplaceTempView("telemetry")

In [47]:
%%sql
select count(*) from telemetry

In [49]:
%%sql
select  day(cast(ts  as timestamp)) as tsDay, 
        hour(cast(ts  as timestamp)) as tsHour,
        avg(pressure) as avgPressure, avg(speed) as avgSpeed
from telemetry
group by  tsDay, tsHour
order by  tsDay, tsHour
